In [1]:
# python imports
import re
import json
from os.path import join
import cPickle
import numpy as np
import math
import itertools

# third party imports\
import pandas
import escher
import cloudpickle

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *
from ecolime import (ribosome, tRNA_charging, transcription, translocation, 
                     chaperones, DNA_replication, translation)
from ecolime.characterization.biomass_allocation import get_membrane_composition, get_membrane_protein

from cobrame import *
from cobrame.util import building, dogma
from cobrame.util.mass import compute_RNA_mass
from cobrame.solve.algorithms import binary_search, fva, solve_at_growth_rate
from cobrame.solve.symbolic import compile_expressions

from cloudpickle import load
import cPickle

from cobrame import ComplexData, TranscribedGene, ModificationData
from cobrame.util.building import add_modification_data

import cobra

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


# import base model

In [2]:
filename = './RNA_sweep_control_prototypes/proto_65_NO_tRNA_mods_NO_rRNA_mods.pickle'
with open(filename, "rb") as infile:
    me = cPickle.load(infile)

In [3]:
import pandas as pd
control_df = pd.DataFrame()
control_df.loc[0,'rRNA'] = True
control_df.loc[1,'rRNA'] = True
control_df.loc[2,'rRNA'] = False
control_df.loc[3,'rRNA'] = False

control_df.loc[0,'tRNA'] = False
control_df.loc[1,'tRNA'] = True
control_df.loc[2,'tRNA'] = False
control_df.loc[3,'tRNA'] = True

control_df


,rRNA,tRNA
0,True,False
1,True,True
2,False,False
3,False,True


In [4]:
def build_control_prototype(rRNA, tRNA):
    """build the controls"""
    
    #input the base model
    filename = './RNA_sweep_control_prototypes/proto_65_NO_tRNA_mods_NO_rRNA_mods.pickle'
    with open(filename, "rb") as infile:
        me = cPickle.load(infile)
    
    #delete the tRNAData
    if not tRNA:
        for trna_rxn in me.reactions.query('charging_tRNA'):
            trna_rxn.tRNAData.modifications = {}
    #
    #
    #
    #
    #
    #add the ribosome with input rRNA = true/false
    ribosome.add_ribosome(me, verbose=False, rRNA_mods = rRNA)
    #RNA_mods = True adds the modifications
    
    # These should be coupled to translation not ribosome formation per iOL1650
    Inf_modifications = ['Translation_initiation_factor_InfA', 'Translation_gtp_initiation_factor_InfB',
                    'Translation_initiation_factor_InfC']
    for sub in Inf_modifications:
        me.complex_data.ribosome.modifications[sub] = 0
        for data in me.translation_data:
            data.modifications[sub]= -1
    
    #me.update()
    #me.prune()
    
    path = './RNA_sweep_control_prototypes/'
    file_name = 'control_proto_65_tRNA_mods_' + str(tRNA) +"_rRNA_mods_" + str(rRNA) + ".pickle"
    with open(path + file_name, "wb") as outfile:
        cPickle.dump(me, outfile)

    

In [5]:
#build the 4 control prototypes
from tqdm import tqdm
for index, row in tqdm(control_df.iterrows()):
    rRNA = row.rRNA
    tRNA = row.tRNA
    build_control_prototype(rRNA, tRNA)
    

4it [00:36,  9.01s/it]
